# Immune Circle Heatmap

This notebooks takes the genes that are a hit from the Brca reactome gsea (metabolism pathway) and maps them on a large circle heat map. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest


import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.utils as u
import plot_utils as p 

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Load df with all of the genes that are FDR significant. Then get list of just the gene names and use them to run a GSEA. 

In [2]:
prot_FDR = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_all_FDR_wide.csv")
df_FDR= prot_FDR.drop(['Unnamed: 0'], axis=1)
df_FDR = df_FDR.set_index("Comparison")
df1_transposed = df_FDR.T 
df1_transposed


Comparison,EGFR_proteomics,PHLDA1_proteomics,GRB2_proteomics,SOCS2_proteomics,CDH4_proteomics,PLA2G15_proteomics,DAB2_proteomics,GLA_proteomics,CTSC_proteomics,CKB_proteomics,...,PHF23_proteomics,RNASEH2A_proteomics,NELFE_proteomics,KIZ_proteomics,ZNF623_proteomics,SLIT3_proteomics,AAK1_proteomics,COMMD3-BMI1_proteomics,LIMA1_proteomics,RFXAP_proteomics
Correlation_Gbm,1.0,8.168478e-01,-6.108891e-01,5.627196e-01,5.591798e-01,-5.566242e-01,-5.564015e-01,-5.504906e-01,-5.462853e-01,5.442463e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P_value_Gbm,0.0,6.553435e-25,1.886384e-11,1.343464e-09,1.790048e-09,2.197562e-09,2.237010e-09,3.570229e-09,4.951742e-09,5.793441e-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Correlation_kidney,1.0,NaN,NaN,NaN,NaN,-2.980294e-01,NaN,-2.983476e-01,-3.023163e-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P_value_kidney,0.0,NaN,NaN,NaN,NaN,1.565619e-03,NaN,1.546704e-03,1.327545e-03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Correlation_Ovar,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P_value_Ovar,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Correlation_Brca,1.0,3.647974e-01,NaN,NaN,NaN,2.741849e-01,3.260553e-01,NaN,2.669396e-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P_value_Brca,0.0,1.902969e-04,NaN,NaN,NaN,3.024564e-03,3.764707e-04,NaN,3.928353e-03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Correlation_Luad,1.0,NaN,-3.024386e-01,NaN,NaN,NaN,NaN,NaN,3.027604e-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P_value_Luad,0.0,NaN,1.321265e-03,NaN,NaN,NaN,NaN,NaN,1.304865e-03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
cptac_prot = df1_transposed.columns.values.tolist()
cptac_genes = []
for gene in cptac_prot :
    cptac_genes.append((re.sub("_proteomics", "", gene)))
len(cptac_genes)

6007

Run GSEA using reactome 2016 set

In [4]:
cptac_enr = gp.enrichr(gene_list = cptac_genes, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='test/enrichr_kegg')
cptac_enr.res2d.head(10)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,Reactome_2016,Metabolism Homo sapiens R-HSA-1430728,861/1908,1.096018e-48,1.676907e-45,0,0,1.502440,165.918060,NUP107;IL4I1;NSDHL;PSMD2;LIPG;PRKACA;PRKACB;XY...
1,Reactome_2016,Hemostasis Homo sapiens R-HSA-109582,306/552,2.574169e-36,1.969239e-33,0,0,1.845673,151.248339,APP;SPARC;SERPINE2;SERPINE1;F11R;ISLR;PPP2R1A;...
2,Reactome_2016,Extracellular matrix organization Homo sapiens...,173/283,1.270343e-27,6.478749e-25,0,0,2.035317,126.048213,SPARC;SERPINE1;COL12A1;ITGA2B;ICAM2;TNC;LOXL3;...
3,Reactome_2016,"Platelet activation, signaling and aggregation...",153/253,6.870135e-24,2.627827e-21,0,0,2.013461,107.387670,DGKG;APP;SPARC;DGKB;DGKA;PROS1;SERPINE1;ITGA2B...
4,Reactome_2016,Immune System Homo sapiens R-HSA-168256,642/1547,1.331768e-23,4.075210e-21,0,0,1.381711,72.778771,ATF1;CYFIP2;NCKAP1;CNTFR;NUP107;WIPF1;WIPF2;WI...
5,Reactome_2016,Signaling by Rho GTPases Homo sapiens R-HSA-19...,201/367,2.396954e-23,6.112233e-21,0,0,1.823486,94.976723,CYFIP2;NCKAP1;TRIO;NUP107;WIPF1;WIPF2;WIPF3;AC...
6,Reactome_2016,Membrane Trafficking Homo sapiens R-HSA-199991,221/420,1.559525e-22,3.408677e-20,0,0,1.751924,87.968484,USP6NL;APP;SCOC;TFRC;GCC2;GCC1;GJA1;GOLGA4;GOL...
7,Reactome_2016,Innate Immune System Homo sapiens R-HSA-168249,369/807,6.881127e-22,1.316016e-19,0,0,1.522387,74.183032,ATF1;CYFIP2;NCKAP1;APP;AHCYL1;WIPF1;WIPF2;WIPF...
8,Reactome_2016,Vesicle-mediated transport Homo sapiens R-HSA-...,248/492,9.098425e-22,1.546732e-19,0,0,1.678259,81.309577,USP6NL;APP;SCOC;SPARC;TFRC;GCC2;GCC1;GJA1;GOLG...
9,Reactome_2016,Metabolism of proteins Homo sapiens R-HSA-392499,464/1074,4.219962e-21,6.456542e-19,0,0,1.438421,67.482753,RPL4;RPL5;RPL30;PGAP1;NUP107;RPL3;RPL34;RPL8;R...


In [5]:
#get just the immune genes
cptac_df = cptac_enr.res2d
immune = cptac_df.iloc[7,9]
immune = immune.split(';')
immune.remove("EGFR")
len(immune)


368

In [6]:

immune2 = ['PSMB5', 'PSMB6', 'PSMB7', 'PSMB8', 'PSMB9', 'PSMB10', 'TAP1', 'TAP2', 'ERAP1', 'ERAP2', 'CANX', 'CALR', 'PDIA3', 'TAPBP', 'B2M', 'HLA-A', 'HLA-B' ,'HLA-C']

In [7]:
#Get append version of the df with all cancer type, fdr sig trans results
df_FDR_append = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_sig_all_prot_append_FDR.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)


In [8]:
#filter down df with just metabolism genes 
immune_column_names = []
for gene in immune2:
    gene += "_proteomics"
    immune_column_names.append(gene)

df_FDR_immune = df_FDR_append[df_FDR_append.Comparison.isin(immune_column_names)]
df_FDR_immune

,Comparison,Correlation,P_value,Cancer Type
1407,PDIA3_proteomics,-0.278904,0.005182,GBM
3049,CALR_proteomics,0.433985,0.000001,Brca
4587,PDIA3_proteomics,0.276275,0.002801,Brca
6463,PDIA3_proteomics,-0.275514,0.003577,Luad
6488,CANX_proteomics,0.273512,0.003838,Luad
7177,PSMB5_proteomics,0.327612,0.000506,Hnscc
7245,PSMB8_proteomics,-0.318132,0.000749,Hnscc
7291,PSMB10_proteomics,-0.311170,0.000990,Hnscc
7294,PSMB7_proteomics,0.310761,0.001006,Hnscc
7463,PSMB9_proteomics,-0.291602,0.002095,Hnscc


In [9]:
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_immune, "P_value","Correlation","Comparison","Cancer Type",plot_width= 500, plot_height = 650,save_png = "immune.png")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

In [10]:
immune2 = ["CD14","C16","CD247","CD64","CD68","ITGAM","AIF1","CCL2","CD163","CD31","CD4","CD8A","FCGR1A","CSF1","CSF1R","CD274","TIM3","IGF1","HLA-DRA","HLA-DRB1","HAVCR2"]
#FCGRA1 = CD64

df_FDR_append_all = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_all_prot_append_return_all.csv")
df_FDR_append_all = df_FDR_append_all.drop(['Unnamed: 0'], axis=1)

In [11]:
immune_column_names = []
for gene in immune2:
    gene += "_proteomics"
    immune_column_names.append(gene)

df_FDR_immune_all = df_FDR_append_all[df_FDR_append_all.Comparison.isin(immune_column_names)]


In [12]:

df_FDR_immune = df_FDR_append[df_FDR_append.Comparison.isin(immune_column_names)]

In [13]:
p.plotCircleHeatMap(df_FDR_immune, "P_value","Correlation","Comparison","Cancer Type",plot_width= 500, plot_height = 650,)

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

In [14]:
p.plotCircleHeatMap(df_FDR_immune_all, "P_value","Correlation","Comparison","Cancer Type",plot_width= 500, plot_height = 650,)

Loading BokehJS ...

In [15]:
prot_FDR = pd.read_csv("../Step3.1_Pearson_dfs_by_cancer/csv_files/Luad_EGFR_all_pearson_FDR.csv")
df_FDR= prot_FDR.drop(['Unnamed: 0'], axis=1)
df_FDR = df_FDR.set_index("Comparison")
df1_transposed = df_FDR.T 
df1_transposed


Comparison,EGFR_proteomics,GGCT_proteomics,LANCL2_proteomics,NUDCD3_proteomics,TAX1BP1_proteomics,ADGRF1_proteomics,GRAP_proteomics,CDK13_proteomics,GRAMD2B_proteomics,MACC1_proteomics,...,WDR19_proteomics,SLC25A22_proteomics,OXSR1_proteomics,PCSK6_proteomics,TMEM205_proteomics,VGLL4_proteomics,ST5_proteomics,CSTF1_proteomics,SPRYD4_proteomics,LDB3_proteomics
Correlation,1.0,6.090062e-01,5.888081e-01,5.683828e-01,5.615545e-01,6.068124e-01,-5.441949e-01,5.413530e-01,5.621980e-01,5.222575e-01,...,-0.261754,0.261729,-0.261669,-0.324513,0.267685,0.261583,0.261481,0.261472,-0.261441,-0.278121
P_value,0.0,1.676601e-12,1.328507e-11,9.362813e-11,1.746498e-10,7.466411e-10,8.008013e-10,1.019153e-09,1.703448e-09,4.866322e-09,...,0.005740,0.005745,0.005756,0.005762,0.005768,0.005773,0.005793,0.005794,0.005801,0.005810


In [16]:
Luad_prot = df1_transposed.columns.values.tolist()
Luad_genes = []
for gene in Luad_prot :
    Luad_genes.append((re.sub("_proteomics", "", gene)))
len(Luad_genes)

1198

In [17]:
Luad_enr = gp.enrichr(gene_list = Luad_genes, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='test/enrichr_kegg')
Luad_enr.res2d.head(5)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,Reactome_2016,Immune System Homo sapiens R-HSA-168256,158/1547,1.186818e-11,1.815831e-08,0,0,1.705061,42.894496,ATF1;CYFIP2;APP;AHCYL1;NCF1;WIPF1;PROS1;WIPF2;...
1,Reactome_2016,Innate Immune System Homo sapiens R-HSA-168249,93/807,8.043351e-10,6.153164e-07,0,0,1.923900,40.288409,ATF1;CYFIP2;APP;AHCYL1;CDKN1B;WIPF1;WIPF2;PROS...
2,Reactome_2016,RHO GTPases Activate WASPs and WAVEs Homo sapi...,14/36,7.762482e-09,3.958866e-06,0,0,6.492302,121.237012,CYFIP2;WIPF1;WIPF2;WAS;BRK1;ARPC4;ARPC5;PTK2;A...
3,Reactome_2016,Hemostasis Homo sapiens R-HSA-109582,66/552,6.232478e-08,2.383923e-05,0,0,1.996080,33.116784,APP;DOCK5;DGKB;PROS1;F13A1;PIK3CD;ARRB2;CLU;AK...
4,Reactome_2016,Membrane Trafficking Homo sapiens R-HSA-199991,54/420,9.540055e-08,2.919257e-05,0,0,2.146435,34.697504,APP;GCC1;USE1;GOLGA5;KIF13B;KIFAP3;VPS36;CTSC;...


In [18]:
#get just the immune genes
luad_df =Luad_enr.res2d
immune = luad_df.iloc[0,9]
immune = immune.split(';')
immune.remove("EGFR")
len(immune)

157

In [19]:
#filter down df with just metabolism genes 
immune_column_names = []
for gene in immune:
    gene += "_proteomics"
    immune_column_names.append(gene)

df_FDR_immune = df_FDR_append[df_FDR_append.Comparison.isin(immune_column_names)]
df_FDR_immune

,Comparison,Correlation,P_value,Cancer Type
2,GRB2_proteomics,-0.610889,1.886384e-11,GBM
8,CTSC_proteomics,-0.546285,4.951742e-09,GBM
25,WAS_proteomics,-0.501918,1.205168e-07,GBM
53,APBB1IP_proteomics,-0.483056,4.103970e-07,GBM
58,CSK_proteomics,-0.481565,4.507321e-07,GBM
...,...,...,...,...
8413,KLC3_proteomics,0.431755,2.483261e-06,Lscc
8431,CDKN1B_proteomics,-0.411578,7.933008e-06,Lscc
8478,CDH1_proteomics,0.385724,3.161296e-05,Lscc
8510,CTNNB1_proteomics,0.369198,7.213703e-05,Lscc


In [20]:
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_immune, "P_value","Correlation","Comparison","Cancer Type",plot_width= 5000, plot_height = 650,save_png = "immune.png")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

In [21]:
prot_FDR = pd.read_csv("../Step3.1_Pearson_dfs_by_cancer/csv_files/Hnscc_EGFR_all_pearson_FDR.csv")
df_FDR= prot_FDR.drop(['Unnamed: 0'], axis=1)
df_FDR = df_FDR.set_index("Comparison")
df1_transposed = df_FDR.T 
df1_transposed


Comparison,EGFR_proteomics,LANCL2_proteomics,PHLDA3_proteomics,PHLDA1_proteomics,PHLDA2_proteomics,TPBG_proteomics,ASRGL1_proteomics,GRB2_proteomics,FSCN1_proteomics,JAG1_proteomics,...,ESF1_proteomics,UTRN_proteomics,NCOA2_proteomics,EPS15_proteomics,APIP_proteomics,GPD1L_proteomics,SELL_proteomics,MBD1_proteomics,RRP12_proteomics,VRK1_proteomics
Correlation,1.0,7.967284e-01,7.637845e-01,6.642711e-01,6.490337e-01,5.508985e-01,-5.324208e-01,-5.323412e-01,5.264348e-01,5.193676e-01,...,0.266758,-0.266757,-0.266724,-0.266711,-0.266625,-0.266473,-0.266360,-0.266098,0.266066,0.266060
P_value,0.0,3.798194e-25,4.542521e-22,3.426615e-15,2.302129e-14,5.408670e-10,2.543321e-09,2.559824e-09,4.117665e-09,7.186589e-09,...,0.005048,0.005049,0.005054,0.005056,0.005071,0.005097,0.005117,0.005162,0.005168,0.005169


In [22]:
Hnscc_prot = df1_transposed.columns.values.tolist()
Hnscc_genes = []
for gene in Hnscc_prot :
    Hnscc_genes.append((re.sub("_proteomics", "", gene)))
len(Hnscc_genes)

1074

In [23]:
Hnscc_enr = gp.enrichr(gene_list = Hnscc_genes, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='test/enrichr_kegg')
Hnscc_enr.res2d.head(10)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,Reactome_2016,Extracellular matrix organization Homo sapiens...,59/283,1.180357e-19,1.805946e-16,0,0,3.882320,169.204302,DDR1;ITGB1;SPARC;ITGB5;SERPINE1;COL12A1;ICAM3;...
1,Reactome_2016,Hemostasis Homo sapiens R-HSA-109582,81/552,1.188696e-16,9.093528e-14,0,0,2.732572,100.199337,ITGB1;SPARC;DOCK8;SERPINE1;PLEK;ATP2A3;ARRB1;A...
2,Reactome_2016,Immune System Homo sapiens R-HSA-168256,158/1547,1.170688e-15,5.970510e-13,0,0,1.901921,65.390299,CYFIP2;NCKAP1;NUP107;NCF1;WIPF1;NCF4;ARAF;INPP...
3,Reactome_2016,Cell surface interactions at the vascular wall...,30/101,5.646039e-15,2.159610e-12,0,0,5.531279,181.469230,ITGB1;F11R;ITGAL;SPN;THBD;INPP5D;BSG;ITGAV;SLC...
4,Reactome_2016,Adaptive Immune System Homo sapiens R-HSA-1280218,92/762,1.973048e-13,6.037528e-11,0,0,2.248322,65.772480,ITGB1;GSK3B;CDKN1B;NCF1;ITGB5;SH3KBP1;CLTC;NCF...
5,Reactome_2016,Signaling by Rho GTPases Homo sapiens R-HSA-19...,55/367,4.616114e-12,1.177109e-09,0,0,2.790759,72.842907,CYFIP2;ARHGAP9;ITGB1;NCKAP1;NUP107;CDKN1B;RTKN...
6,Reactome_2016,Metabolism Homo sapiens R-HSA-1430728,170/1908,1.454692e-11,3.179542e-09,0,0,1.659191,41.402846,RPL4;ALDH1L1;RPL30;NUP107;RPL3;INPPL1;MT1X;ENO...
7,Reactome_2016,"Platelet activation, signaling and aggregation...",42/253,6.104470e-11,1.167480e-08,0,0,3.091395,72.707799,SPARC;SERPINE1;PLEK;ITPR1;ITPR3;ARRB1;ARRB2;RA...
8,Reactome_2016,Collagen formation Homo sapiens R-HSA-1474290,23/85,6.925813e-11,1.177388e-08,0,0,5.038887,117.875595,COL17A1;COL28A1;COL15A1;CRTAP;LAMB3;COL11A1;CO...
9,Reactome_2016,VEGFA-VEGFR2 Pathway Homo sapiens R-HSA-4420097,48/320,1.039101e-10,1.589824e-08,0,0,2.793296,64.210881,CYFIP2;NCKAP1;NCF1;NCF4;ARAF;ITPR1;HSPB1;ITPR3...


In [31]:
#get just the immune genes
Hnscc_df =Hnscc_enr.res2d
immune = Hnscc_df.iloc[4,9]
immune = immune.split(';')
immune.remove("EGFR")
len(immune)

91

In [32]:
#filter down df with just metabolism genes 
immune_column_names = []
for gene in immune:
    gene += "_proteomics"
    immune_column_names.append(gene)

df_FDR_immune = df_FDR_append[df_FDR_append.Comparison.isin(immune_column_names)]
df_FDR_immune

,Comparison,Correlation,P_value,Cancer Type
2,GRB2_proteomics,-0.610889,1.886384e-11,GBM
25,WAS_proteomics,-0.501918,1.205168e-07,GBM
58,CSK_proteomics,-0.481565,4.507321e-07,GBM
73,PLCG2_proteomics,-0.472453,7.918099e-07,GBM
75,SYK_proteomics,-0.470849,8.729180e-07,GBM
...,...,...,...,...
8331,NCF1_proteomics,0.291873,3.722237e-03,Colon
8431,CDKN1B_proteomics,-0.411578,7.933008e-06,Lscc
8556,FBXO3_proteomics,0.350610,1.732373e-04,Lscc
8665,ITGAV_proteomics,0.317536,7.242515e-04,Lscc


In [33]:
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_immune, "P_value","Correlation","Comparison","Cancer Type",plot_width= 5000, plot_height = 650,save_png = "immune.png")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

In [27]:
df2 = u.search_reactome_proteins_in_pathways("R-HSA-2132295.3")
immune_genes = df2.member.unique()

immune_all_genes = []
for gene in immune_genes:
    gene += "_proteomics"
    immune_all_genes.append(gene)

In [28]:
df2

,pathway_id,member
0,R-HSA-2132295.3,ACTR10
1,R-HSA-2132295.3,ACTR1A
2,R-HSA-2132295.3,ACTR1B
3,R-HSA-2132295.3,AP1B1
4,R-HSA-2132295.3,AP1G1
...,...,...
122,R-HSA-2132295.3,TUBB4A
123,R-HSA-2132295.3,TUBB4B
124,R-HSA-2132295.3,TUBB6
125,R-HSA-2132295.3,TUBB8


In [34]:
df_FDR_immune = df_FDR_append_all[df_FDR_append_all.Comparison.isin(immune_column_names)]
df_FDR_immune

,Comparison,Correlation,P_value,Cancer Type
2,GRB2_proteomics,-0.610889,1.886384e-11,GBM
25,WAS_proteomics,-0.501918,1.205168e-07,GBM
58,CSK_proteomics,-0.481565,4.507321e-07,GBM
73,PLCG2_proteomics,-0.472453,7.918099e-07,GBM
75,SYK_proteomics,-0.470849,8.729180e-07,GBM
...,...,...,...,...
19734,SH3KBP1_proteomics,-0.203521,3.296251e-02,Lscc
19741,ITGAL_proteomics,-0.203032,3.339509e-02,Lscc
19840,PSMD2_proteomics,0.197970,3.815582e-02,Lscc
19869,BTK_proteomics,-0.196563,3.957581e-02,Lscc


In [35]:
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_immune, "P_value","Correlation","Comparison","Cancer Type",plot_width= 5000, plot_height = 650,save_png = "immune.png")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...